# Summary Buffer Memory

## Colab-specific setup

Make sure you have a Database and get ready to upload the Secure Connect Bundle and supply the Token string
(see [Pre-requisites](https://cassio.org/start_here/#vector-database) on cassio.org for details).

Likewise, ensure you have the necessary secret for the LLM provider of your choice: you'll be asked to input it shortly
(see [Pre-requisites](https://cassio.org/start_here/#llm-access) on cassio.org for details).

_Note: this notebook is part of the CassIO documentation. Visit [this page on cassIO.org](https://cassio.org/frameworks/langchain/memory-summarybuffermemory/)._


In [ ]:
# install required dependencies
! pip install \
    "git+https://github.com/hemidactylus/langchain@cassio-full-preview#egg=langchain" \
    "cassandra-driver>=3.28.0" \
    "cassio==0.0.6" \
    "google-cloud-aiplatform>=1.25.0" \
    "jupyter>=1.0.0" \
    "openai==0.27.7" \
    "python-dotenv==1.0.0" \
    "tensorflow-cpu==2.12.0" \
    "tiktoken==0.4.0" \
    "transformers>=4.29.2"

You will likely be asked to "Restart the Runtime" at this time, as some dependencies
have been upgraded. **Please do restart the runtime now** for a smoother execution from this point onward.

In [ ]:
# Input your database keyspace name:
ASTRA_DB_KEYSPACE = input('Your Astra DB Keyspace name: ')

In [ ]:
# Input your Astra DB token string, the one starting with "AstraCS:..."
ASTRA_DB_TOKEN_BASED_PASSWORD = input('Your Astra DB Token: ')

### Astra DB Secure Connect Bundle

Please upload the Secure Connect Bundle zipfile to connect to your Astra DB instance.

The Secure Connect Bundle is needed to establish a secure connection to the database.
Click [here](https://awesome-astra.github.io/docs/pages/astra/download-scb/#c-procedure) for instructions on how to download it from Astra DB.

In [ ]:
# Upload your Secure Connect Bundle zipfile:
import os
from google.colab import files


print('Please upload your Secure Connect Bundle')
uploaded = files.upload()
if uploaded:
    astraBundleFileTitle = list(uploaded.keys())[0]
    ASTRA_DB_SECURE_BUNDLE_PATH = os.path.join(os.getcwd(), astraBundleFileTitle)
else:
    raise ValueError(
        'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
    )

In [ ]:
# colab-specific override of helper functions
from cassandra.cluster import (
    Cluster,
)
from cassandra.auth import PlainTextAuthProvider

# The "username" is the literal string 'token' for this connection mode:
ASTRA_DB_TOKEN_BASED_USERNAME = 'token'


def getCQLSession(mode='astra_db'):
    if mode == 'astra_db':
        cluster = Cluster(
            cloud={
                "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
            },
            auth_provider=PlainTextAuthProvider(
                ASTRA_DB_TOKEN_BASED_USERNAME,
                ASTRA_DB_TOKEN_BASED_PASSWORD,
            ),
        )
        astraSession = cluster.connect()
        return astraSession
    else:
        raise ValueError('Unsupported CQL Session mode')

def getCQLKeyspace(mode='astra_db'):
    if mode == 'astra_db':
        return ASTRA_DB_KEYSPACE
    else:
        raise ValueError('Unsupported CQL Session mode')

### LLM Provider

In the cell below you can choose between **GCP VertexAI** or **OpenAI** for your LLM services.
(See [Pre-requisites](https://cassio.org/start_here/#llm-access) on cassio.org for more details).

Make sure you set the `llmProvider` variable and supply the corresponding access secrets in the following cell.

In [ ]:
# Set your secret(s) for LLM access:
llmProvider = 'OpenAI'  # 'GCP_VertexAI', 'Azure_OpenAI'


In [ ]:
if llmProvider == 'OpenAI':
    apiSecret = input(f'Your secret for LLM provider "{llmProvider}": ')
    os.environ['OPENAI_API_KEY'] = apiSecret
elif llmProvider == 'GCP_VertexAI':
    # we need a json file
    print(f'Please upload your Service Account JSON for the LLM provider "{llmProvider}":')
    from google.colab import files
    uploaded = files.upload()
    if uploaded:
        vertexAIJsonFileTitle = list(uploaded.keys())[0]
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(os.getcwd(), vertexAIJsonFileTitle)
    else:
        raise ValueError(
            'No file uploaded. Please re-run the cell.'
        )
elif llmProvider == 'Azure_OpenAI':
    # a few parameters must be input
    apiSecret = input(f'Your API Key for LLM provider "{llmProvider}": ')
    os.environ['AZURE_OPENAI_API_KEY'] = apiSecret
    apiBase = input('The "Base URL" for your models (e.g. "https://YOUR-RESOURCE-NAME.openai.azure.com"): ')
    os.environ['AZURE_OPENAI_API_BASE'] = apiBase
    apiLLMDepl = input('The name of your LLM Deployment: ')
    os.environ['AZURE_OPENAI_LLM_DEPLOYMENT'] = apiLLMDepl
    apiLLMModel = input('The name of your LLM Model (e.g. "gpt-4"): ')
    os.environ['AZURE_OPENAI_LLM_MODEL'] = apiLLMModel
    apiEmbDepl = input('The name for your Embeddings Deployment: ')
    os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT'] = apiEmbDepl
    apiEmbModel = input('The name of your Embedding Model (e.g. "text-embedding-ada-002"): ')
    os.environ['AZURE_OPENAI_EMBEDDINGS_MODEL'] = apiEmbModel

    # The following is probably not going to change for some time...
    os.environ['AZURE_OPENAI_API_VERSION'] = '2023-03-15-preview'
else:
    raise ValueError('Unknown/unsupported LLM Provider')

### Colab preamble completed

The following cells constitute the demo notebook proper.

# Summary Buffer Memory

The same `CassandraChatMessageHistory` can power a more sophisticated kind of chat memory that automatically maintains a **summary** of the past interactions, so as to provide a memory that virtually extends arbitrarily far back in the past while fitting a finite amount of tokens.

## Notice

**this demo is currently not working with the Azure OpenAI LLMs. Please run it with another LLM provider.**

In [1]:
from langchain.memory import CassandraChatMessageHistory
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain

In [2]:
# creation of the DB connection
cqlMode = 'astra_db'
session = getCQLSession(mode=cqlMode)
keyspace = getCQLKeyspace(mode=cqlMode)

In [3]:
message_history = CassandraChatMessageHistory(
    session_id='summarized-conversation-4321',
    session=session,
    keyspace=keyspace,
    ttl_seconds = 3600,
)

In [4]:
message_history.clear()

Below is the logic to instantiate the LLM of choice. We choose to leave it in the notebooks for clarity.

In [5]:
import os
# creation of the LLM resources


if llmProvider == 'GCP_VertexAI':
    from langchain.llms import VertexAI
    llm = VertexAI()
    print('LLM from VertexAI')
elif llmProvider == 'OpenAI':
    os.environ['OPENAI_API_TYPE'] = 'open_ai'
    from langchain.llms import OpenAI
    llm = OpenAI()
    print('LLM from OpenAI')
elif llmProvider == 'Azure_OpenAI':
    os.environ['OPENAI_API_TYPE'] = 'azure'
    os.environ['OPENAI_API_VERSION'] = os.environ['AZURE_OPENAI_API_VERSION']
    os.environ['OPENAI_API_BASE'] = os.environ['AZURE_OPENAI_API_BASE']
    os.environ['OPENAI_API_KEY'] = os.environ['AZURE_OPENAI_API_KEY']
    from langchain.llms import AzureOpenAI
    llm = AzureOpenAI(temperature=0, model_name=os.environ['AZURE_OPENAI_LLM_MODEL'],
                      engine=os.environ['AZURE_OPENAI_LLM_DEPLOYMENT'])
    print('LLM from Azure OpenAI')
else:
    raise ValueError('Unknown LLM provider.')

LLM from OpenAI


## Use the memory with summary in a Chain

In [6]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    chat_memory=message_history,
    max_token_limit=180,
)

In [7]:
summaryConversation = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=True
)

With the verbose output you can track the summary. It starts empty and once few messages have been exchanged it starts to "incorporate" those messages that would have been lost in the past.

In [8]:
summaryConversation.predict(input='Tell me about William Tell, the hero')



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Tell me about William Tell, the hero
AI:

> Finished chain.


" William Tell was a legendary Swiss hero who lived in the late 13th or early 14th century. He is most known for his feat of shooting an apple off of his son's head with a crossbow. This act was said to be an act of defiance against a tyrannical Austrian ruler, Gessler, who had placed his hat atop a pole in the town square and demanded that all citizens bow to it. Tell's refusal to do so earned him his place in Swiss folklore."

In [9]:
summaryConversation.predict(
    input="Any relation to the apple from Cupertino?"
)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Tell me about William Tell, the hero
AI:  William Tell was a legendary Swiss hero who lived in the late 13th or early 14th century. He is most known for his feat of shooting an apple off of his son's head with a crossbow. This act was said to be an act of defiance against a tyrannical Austrian ruler, Gessler, who had placed his hat atop a pole in the town square and demanded that all citizens bow to it. Tell's refusal to do so earned him his place in Swiss folklore.
Human: Any relation to the apple from Cupertino?
AI:

> Finished chain.


'I do not know.'

No messages have slided past the "recent window" yet -- correspondingly, the summary buffer is still empty:

In [10]:
memory.moving_summary_buffer

''

Continue the conversation - you might start to see the summary of past exchanges making its way to the final prompt for the LLM in the form of a "System" contribution to the _Current conversation_:

In [11]:
summaryConversation.predict(
    input="I heard the two apples may be in fact the very same fruit."
)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Tell me about William Tell, the hero
AI:  William Tell was a legendary Swiss hero who lived in the late 13th or early 14th century. He is most known for his feat of shooting an apple off of his son's head with a crossbow. This act was said to be an act of defiance against a tyrannical Austrian ruler, Gessler, who had placed his hat atop a pole in the town square and demanded that all citizens bow to it. Tell's refusal to do so earned him his place in Swiss folklore.
Human: Any relation to the apple from Cupertino?
AI: I do not know.
Human: I heard the two apples may be in fact the very same fruit.
AI:

> Finished chain.


" That seems highly unlikely given that William Tell's feat is said to have happened in the 13th or 14th century, while the Apple from Cupertino was founded in the late 20th century."

This is where the current summary buffer is stored

_(Note: depending on the length of the messages so far, at this point you might or might not see an empty summary. In any case, keep reading and it will doubtless start to be nonempty.)_

In [12]:
memory.moving_summary_buffer

'\nThe human asks the AI to tell them about William Tell, the hero.'

The whole conversation is still in the "recent" window. Below is what will start to happen in the `ConversationSummaryBufferMemory` at the next chat message: starting from the message list, and the summary so far (if any), the LLM is asked to come up with a new summary which will overwrite the current one, stored in `memory.moving_summary_buffer`.

In [13]:
print(memory.predict_new_summary(
    memory.chat_memory.messages,
    memory.moving_summary_buffer,
))


The human asked the AI to tell them about William Tell, the hero. The AI explained that Tell was a legendary Swiss hero who was known for shooting an apple off of his son's head with a crossbow to defy a tyrannical Austrian ruler, Gessler. The human then asked if there was any relation to the Apple from Cupertino, to which the AI replied that it was unlikely given that Tell's feat happened centuries before the company was founded.


In [14]:
summaryConversation.predict(
    input='Ok, enough with apples. Let\'s talk starfish now.'
)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human asks the AI to tell them about William Tell, the hero.
Human: Tell me about William Tell, the hero
AI:  William Tell was a legendary Swiss hero who lived in the late 13th or early 14th century. He is most known for his feat of shooting an apple off of his son's head with a crossbow. This act was said to be an act of defiance against a tyrannical Austrian ruler, Gessler, who had placed his hat atop a pole in the town square and demanded that all citizens bow to it. Tell's refusal to do so earned him his place in Swiss folklore.
Human: Any relation to the apple from Cupertino?
AI: I do not know.
Human: I heard the two apples may be in fact the very same fruit.
AI:  Tha

" Sure! Starfish, or sea stars, are a group of echinoderms found in all of the world's oceans. They have five arms and a central disc, and they range in size from a few centimeters to over a meter in diameter. Starfish are found in a variety of habitats, from shallow tidal pools to deep sea floors. They are carnivorous predators and feed on mollusks, crustaceans, and other small organisms."

In [15]:
memory.moving_summary_buffer

"\nThe human asks the AI to tell them about William Tell, the hero. The AI explains that William Tell was a legendary Swiss hero from the late 13th or early 14th century, who is most known for shooting an apple off his son's head with a crossbow. This act was an act of defiance against an Austrian ruler, Gessler, who had placed his hat atop a pole in the town square and demanded that all citizens bow to it. The human then jokes about any relation to the apple from Cupertino."

Now the chat history has presumably reached a certain threshold and there is a summary. In the verbose output below, you will see it appearing in the prompt as a "System" role in the conversation recap.

In [16]:
summaryConversation.predict(
    input='I hear that some of those destroy marine ecosystems.'
)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human asks the AI to tell them about William Tell, the hero. The AI explains that William Tell was a legendary Swiss hero from the late 13th or early 14th century, who is most known for shooting an apple off his son's head with a crossbow. This act was an act of defiance against an Austrian ruler, Gessler, who had placed his hat atop a pole in the town square and demanded that all citizens bow to it. The human then jokes about any relation to the apple from Cupertino.
Human: Tell me about William Tell, the hero
AI:  William Tell was a legendary Swiss hero who lived in the late 13th or early 14th century. He is most known for his feat of shooting an apple off of his son's h

' Yes, that is true. Some species of starfish, notably the crown-of-thorns starfish, can be invasive and destructive to coral reef ecosystems. The starfish feed on the coral, often causing catastrophic damage to the reef.'

In [17]:
memory.moving_summary_buffer

"\nThe human asks the AI to tell them about William Tell, the hero. The AI explains that William Tell was a legendary Swiss hero from the late 13th or early 14th century, who is most known for shooting an apple off his son's head with a crossbow. This act was an act of defiance against an Austrian ruler, Gessler, who had placed his hat atop a pole in the town square and demanded that all citizens bow to it. The human then jokes about any relation to the apple from Cupertino, to which the AI responds that it is highly unlikely given the timeline of their respective histories."

From this point on, the summary will not grow indefinitely in size, but will rather try to cover a longer and longer conversation in a fixed amount of text.

In [18]:
print(memory.predict_new_summary(
    memory.chat_memory.messages,
    memory.moving_summary_buffer,
))


The human asks the AI to tell them about William Tell, the hero. The AI explains that William Tell was a legendary Swiss hero from the late 13th or early 14th century, who is most known for shooting an apple off his son's head with a crossbow as an act of defiance against a tyrannical Austrian ruler. The human then jokes about any relation to the apple from Cupertino, to which the AI responds that it is highly unlikely given the timeline of their respective histories. The human then shifts the conversation to starfish, to which the AI explains that they are a group of echinoderms found in all the world's oceans. The AI also mentions that some species of starfish, such as the crown-of-thorns starfish, can be invasive and destructive to coral reef ecosystems.


## What now?

This demo is hosted [here](https://cassio.org/frameworks/langchain/memory-summarybuffermemory/) at cassio.org.

Discover the other ways you can integrate 
Cassandra/Astra DB with your ML/GenAI needs,
right **within [your favorite framework](https://cassio.org/frameworks/langchain/about/)**.